In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/DC_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,7727377003,DC,"702 H Street NW, WASHINGTON, DC",38.899623,-77.022221,1.100101e+14
1,8934987202,DC,"2400 N Street, NW, Washington, DC",38.907031,-77.052354,1.100101e+14
2,7310038410,DC,"1100 New Jersey Ave SE, Washington, DC",38.877044,-77.003816,1.100101e+14
3,7754307206,DC,"1776 K ST, WASHINGTON, DC",38.902123,-77.041326,1.100101e+14
4,9833567004,DC,"1100 13TH ST NW, WASHINGTON, DC",38.904085,-77.030089,1.100101e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,7727377003,DC,"702 H Street NW, WASHINGTON, DC",38.899623,-77.022221,1.100101e+14
1,8934987202,DC,"2400 N Street, NW, Washington, DC",38.907031,-77.052354,1.100101e+14
2,7310038410,DC,"1100 New Jersey Ave SE, Washington, DC",38.877044,-77.003816,1.100101e+14
3,7754307206,DC,"1776 K ST, WASHINGTON, DC",38.902123,-77.041326,1.100101e+14
4,9833567004,DC,"1100 13TH ST NW, WASHINGTON, DC",38.904085,-77.030089,1.100101e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
469,4189318306,DC,"106 Air Cargo Rd, Washington, DC",38.843642,-77.043856,NaN
1101,9376588108,DC,"1100 15TH ST NW STE 400, WASHINGTON, DC",38.903974,-77.034952,NaN
1687,5791017300,DC,"1995 W 5TH ST, WASHINGTON, DC",35.574706,-77.085720,NaN
2235,7241587203,DC,"PO Box 92592, WASHINGTON, DC",33.507399,-117.123579,NaN
2452,5693987109,DC,"5101 WISCONSIN AVE UNIT 102, WASHINGTON, DC",38.962670,-77.086822,NaN
3276,6713817303,DC,"1801 Coral Way Ste 312, WASHINGTON, DC",25.750967,-80.224109,NaN
3337,6590707010,DC,"1 aviation drive, WASHINGTON, DC",38.858413,-77.044853,NaN
3700,8221899006,DC,"2393 Watters Glen Ct.2420 New York Ave. NE, Wa...",38.894416,-77.209191,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('11')] #DC
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/8 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

8


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_14015/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
469,4189318306,DC,"106 Air Cargo Rd, Washington, DC",38.843642,-77.043856,[]
1101,9376588108,DC,"1100 15TH ST NW STE 400, WASHINGTON, DC",38.903974,-77.034952,[110010101001002]
1687,5791017300,DC,"1995 W 5TH ST, WASHINGTON, DC",35.574706,-77.085720,[]
2235,7241587203,DC,"PO Box 92592, WASHINGTON, DC",33.507399,-117.123579,[]
2452,5693987109,DC,"5101 WISCONSIN AVE UNIT 102, WASHINGTON, DC",38.962670,-77.086822,[]
3276,6713817303,DC,"1801 Coral Way Ste 312, WASHINGTON, DC",25.750967,-80.224109,[]
3337,6590707010,DC,"1 aviation drive, WASHINGTON, DC",38.858413,-77.044853,[]
3700,8221899006,DC,"2393 Watters Glen Ct.2420 New York Ave. NE, Wa...",38.894416,-77.209191,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_14015/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
469,4189318306,DC,"106 Air Cargo Rd, Washington, DC",38.843642,-77.043856,NaN
1101,9376588108,DC,"1100 15TH ST NW STE 400, WASHINGTON, DC",38.903974,-77.034952,110010101001002
1687,5791017300,DC,"1995 W 5TH ST, WASHINGTON, DC",35.574706,-77.085720,NaN
2235,7241587203,DC,"PO Box 92592, WASHINGTON, DC",33.507399,-117.123579,NaN
2452,5693987109,DC,"5101 WISCONSIN AVE UNIT 102, WASHINGTON, DC",38.962670,-77.086822,NaN
3276,6713817303,DC,"1801 Coral Way Ste 312, WASHINGTON, DC",25.750967,-80.224109,NaN
3337,6590707010,DC,"1 aviation drive, WASHINGTON, DC",38.858413,-77.044853,NaN
3700,8221899006,DC,"2393 Watters Glen Ct.2420 New York Ave. NE, Wa...",38.894416,-77.209191,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
469,4189318306,DC,"106 Air Cargo Rd, Washington, DC",38.843642,-77.043856,NaN
1687,5791017300,DC,"1995 W 5TH ST, WASHINGTON, DC",35.574706,-77.085720,NaN
2235,7241587203,DC,"PO Box 92592, WASHINGTON, DC",33.507399,-117.123579,NaN
2452,5693987109,DC,"5101 WISCONSIN AVE UNIT 102, WASHINGTON, DC",38.962670,-77.086822,NaN
3276,6713817303,DC,"1801 Coral Way Ste 312, WASHINGTON, DC",25.750967,-80.224109,NaN
3337,6590707010,DC,"1 aviation drive, WASHINGTON, DC",38.858413,-77.044853,NaN
3700,8221899006,DC,"2393 Watters Glen Ct.2420 New York Ave. NE, Wa...",38.894416,-77.209191,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
469,4189318306,NaN
1101,9376588108,110010101001002
1687,5791017300,NaN
2235,7241587203,NaN
2452,5693987109,NaN
3276,6713817303,NaN
3337,6590707010,NaN
3700,8221899006,NaN


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/DC_fips_scraped.csv") 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,196844,7727377003,DC,"702 H Street NW, WASHINGTON, DC",38.899623,-77.022221,1.100101e+14
1,196845,8934987202,DC,"2400 N Street, NW, Washington, DC",38.907031,-77.052354,1.100101e+14
2,196846,7310038410,DC,"1100 New Jersey Ave SE, Washington, DC",38.877044,-77.003816,1.100101e+14
3,196847,7754307206,DC,"1776 K ST, WASHINGTON, DC",38.902123,-77.041326,1.100101e+14
4,196848,9833567004,DC,"1100 13TH ST NW, WASHINGTON, DC",38.904085,-77.030089,1.100101e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,196844,7727377003,DC,"702 H Street NW, WASHINGTON, DC",38.899623,-77.022221,1.100101e+14,NaN
1,196845,8934987202,DC,"2400 N Street, NW, Washington, DC",38.907031,-77.052354,1.100101e+14,NaN
2,196846,7310038410,DC,"1100 New Jersey Ave SE, Washington, DC",38.877044,-77.003816,1.100101e+14,NaN
3,196847,7754307206,DC,"1776 K ST, WASHINGTON, DC",38.902123,-77.041326,1.100101e+14,NaN
4,196848,9833567004,DC,"1100 13TH ST NW, WASHINGTON, DC",38.904085,-77.030089,1.100101e+14,NaN
...,...,...,...,...,...,...,...,...
4378,201222,7366288501,DC,"2401 Shannon Pl SE, Washington, DC",38.863633,-76.993209,1.100101e+14,NaN
4379,201223,7892588508,DC,"1205 28th St NW, Washington, DC",38.905472,-77.056874,1.100100e+14,NaN
4380,201224,8183137301,DC,"2141 K ST NW STE 607, WASHINGTON, DC",38.902905,-77.047987,1.100101e+14,NaN
4381,201225,9153308500,DC,"700 Monroe St NE, Washington, DC",38.932611,-76.995942,1.100101e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,196844,7727377003,DC,"702 H Street NW, WASHINGTON, DC",38.899623,-77.022221,1.100101e+14,NaN,110010058025008.0
1,196845,8934987202,DC,"2400 N Street, NW, Washington, DC",38.907031,-77.052354,1.100101e+14,NaN,110010055032003.0
2,196846,7310038410,DC,"1100 New Jersey Ave SE, Washington, DC",38.877044,-77.003816,1.100101e+14,NaN,110010072031004.0
3,196847,7754307206,DC,"1776 K ST, WASHINGTON, DC",38.902123,-77.041326,1.100101e+14,NaN,110010107001015.0
4,196848,9833567004,DC,"1100 13TH ST NW, WASHINGTON, DC",38.904085,-77.030089,1.100101e+14,NaN,110010101003000.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,7727377003,DC,"702 H Street NW, WASHINGTON, DC",38.899623,-77.022221,110010058025008.0
1,8934987202,DC,"2400 N Street, NW, Washington, DC",38.907031,-77.052354,110010055032003.0
2,7310038410,DC,"1100 New Jersey Ave SE, Washington, DC",38.877044,-77.003816,110010072031004.0
3,7754307206,DC,"1776 K ST, WASHINGTON, DC",38.902123,-77.041326,110010107001015.0
4,9833567004,DC,"1100 13TH ST NW, WASHINGTON, DC",38.904085,-77.030089,110010101003000.0
...,...,...,...,...,...,...
4378,7366288501,DC,"2401 Shannon Pl SE, Washington, DC",38.863633,-76.993209,110010074011015.0
4379,7892588508,DC,"1205 28th St NW, Washington, DC",38.905472,-77.056874,110010001011003.0
4380,8183137301,DC,"2141 K ST NW STE 607, WASHINGTON, DC",38.902905,-77.047987,110010107001007.0
4381,9153308500,DC,"700 Monroe St NE, Washington, DC",38.932611,-76.995942,110010092011000.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
469,4189318306,DC,"106 Air Cargo Rd, Washington, DC",38.843642,-77.043856,NaN
1687,5791017300,DC,"1995 W 5TH ST, WASHINGTON, DC",35.574706,-77.085720,NaN
2235,7241587203,DC,"PO Box 92592, WASHINGTON, DC",33.507399,-117.123579,NaN
2452,5693987109,DC,"5101 WISCONSIN AVE UNIT 102, WASHINGTON, DC",38.962670,-77.086822,NaN
3276,6713817303,DC,"1801 Coral Way Ste 312, WASHINGTON, DC",25.750967,-80.224109,NaN
3337,6590707010,DC,"1 aviation drive, WASHINGTON, DC",38.858413,-77.044853,NaN
3700,8221899006,DC,"2393 Watters Glen Ct.2420 New York Ave. NE, Wa...",38.894416,-77.209191,NaN


Convert to File

In [19]:
fips_DC = fips_merge_drop

In [20]:
fips_DC.to_csv("../../../data/state_data/geo/geo_fips/23/DC_fips_scraped.csv") 